In [1]:
import torch
from torch import nn




In [2]:
class MLP(nn.Module):
    def __init__(self, **kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10) # 输出层
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0382, -0.0484,  0.0613, -0.0114, -0.2631, -0.1062, -0.2776, -0.0144,
          0.1924, -0.2814],
        [-0.1235, -0.0480,  0.0632, -0.0682, -0.1730, -0.3258, -0.1759, -0.0529,
          0.0943, -0.2443]], grad_fn=<AddmmBackward>)

In [3]:
# class MySequential(nn.Module):
#     from collections import OrderedDict
#     def __init__(self, *args):
#         if len(args) == 1 and isinstance(arg[0], OrderedDict):
#             for key, module in args[0].items():
#                 self.add_module(key, module)
#         else:
#             for idx, module in enumerate(args):
#                 self.add_module(str(idx), module)
#     def forward(self,input):
#         for module in self._modules.values():
#             input = module(input)
#         return input

class MySequential(nn.Module):
    
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传⼊的是⼀个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module) # add_module⽅法会将module添加进self._modules(⼀个OrderedDict)
        else: # 传⼊的是⼀些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
    # self._modules返回⼀个 OrderedDict，保证会按照成员添加时的顺序遍历成
        for module in self._modules.values():
            input = module(input)
        return input

In [4]:
net = MySequential(
nn.Linear(784, 256),
nn.ReLU(),
nn.Linear(256, 10),
 )
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0526, -0.2935,  0.3236, -0.1290,  0.1691,  0.0055, -0.0685, -0.0303,
          0.1282, -0.0395],
        [ 0.0333, -0.3207,  0.2743, -0.1516,  0.2288, -0.0022,  0.0512,  0.0617,
          0.0990, -0.1967]], grad_fn=<AddmmBackward>)

In [12]:
class FancyMLP(nn.Module):
    def __init__(self, **args):
        super(FancyMLP,self).__init__(**args)
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20,20)
        
    def forward(self, x):
        x = self.linear(x)
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data)+1)
        x = self.linear(x)
        while x.norm().item() > 1:
            x/=2
        if x.norm().item() < 0.8:
            x*=10
        return x.sum()

In [13]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(1.6901, grad_fn=<SumBackward0>)

In [14]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())
    def forward(self, x):
        return self.net(x)
net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())
X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(11.9953, grad_fn=<SumBackward0>)